In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import curve_fit
import pandas as pd

plt.style.use(hep.style.ROOT)

In [ ]:
#random trigger data
ptrigDF = pd.read_pickle(r'./PTRIG_LG50_HG50_4k_R28.pkl')
#cosmic or beam data
df = pd.read_pickle(r'./BEAM_LG50_HG50_4k_R27.pkl')
#set to 1 for 4k files, 2 for 8k files
ADCmult = 1

In [ ]:
#plot random trigger spectra for each low gain channel. Fit to gaussian and record pedestals for cuts later

fig, axs=plt.subplots(4, 10, figsize=(0.5*len(ptrigDF.columns),15),sharey=True,sharex=True)
i=0
pedMeans = []
pedStds = []

def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))


for i in range(round(len(ptrigDF.columns)/2)):
    if i%4 == 0: 
        print(i/4)
    plt.sca(axs[i%4][i//4])
    
    plt.rcParams['figure.facecolor']='white'
    plt.rcParams['savefig.facecolor']='white'
    plt.rcParams['savefig.bbox']='tight'
    hist, bin_edges = np.histogram(np.array(getattr(ptrigDF,"Ch_{}_LG".format("{:02d}".format(i)))),range=(0,100),bins=100)
   
    bin_centres = (bin_edges[:-1] + bin_edges[1:])/2
    
    
    p0 = [max(hist), sum(getattr(ptrigDF,"Ch_{}_LG".format("{:02d}".format(i))))/len(getattr(ptrigDF,"Ch_{}_LG".format("{:02d}".format(i)))), 3.]
    coeff, var_matrix = curve_fit(gauss, bin_centres, hist, p0=p0)
    hist_fit = gauss(np.linspace(0,100,5000), *coeff)
    plt.errorbar(bin_centres, y=hist, yerr=np.sqrt(hist),fmt='o')
    plt.plot(np.linspace(0,100,5000), hist_fit, label='$\mu$=%2.0f Chs, \n$\sigma=$%2.0f Chs'%(coeff[1],abs(coeff[2])))
 
    pedMeans.append(coeff[1])

    pedStds.append(abs(coeff[2]))
    #print('Resolution = %2.1f '%(100*coeff[2]/coeff[1]))

    plt.xlabel("ADC Units")
    plt.ylabel("Counts")
  
    plt.title("LG Ch {}".format("{:02d}".format(i)))
    plt.tight_layout()
    plt.legend(fontsize=17,loc='upper right')

In [3]:
df = pd.read_pickle(r'./BEAM_LG50_HG50_4k_R27.pkl')

eventPath = '/Users/spreins/Downloads/Events.txt'
with open(eventPath, 'a') as f:
    f.write("")
    
mips = [52.74750277855359, 48.30860848773839, 53.943404039851835, 44.92190677955567, 61.26714324627709, 65.86727404494636,
50.49966238440961, 55.02522837553596, 46.61446015527332, 60.39973772203866, 55.30173491280392, 62.707398178412745,
50.482420120919734, 62.40987646935536, 59.02966599638661, 55.30173491280392, 123.13587491197416, 118.85827154485337,
93.36027551977364, 104.55347745998186, 88.72268957494947, 87.9317698337502, 99.4048611423608, 109.98290193288523,
102.39942115610283, 82.29288341793296, 93.36027551977364, 93.1310966604848, 120.5428145522235, 96.02936108638767,
107.29564320373284, 93.36027551977364, 78.16073077714876, 62.53205599751284, 90.81216227164441, 90.71800272849238,
109.00527397085415, 109.68782873235487, 85.36866495961932, 93.52544458950403]
mips = [i * ADCmult for i in mips]


for evt in range(10):    
    with open(eventPath, 'a') as f:
        f.write('\n')
        f.write("Event")
        f.write('\n');
    zpos = 2.425
    xpos = 0
    ypos = 0
    #cm
    for layer in range(10):
        
        for ch in range(4):
            
        if ch == 0:
            xpos = -2.4
            ypos = 2.4
        if ch == 1:
            xpos == 2.4
            ypos == 2.4
        if ch == 2:
            xpos = -2.4
            ypos = -2.4
        if ch == 3:
            xpos = 2.4
            ypos = -2.4
        
        energy = getattr(df,"Ch_{}_LG".format("{:02d}".format(ch)))[evtn] - pedMeans[ch]
        
        if energy >= (pedMeans[ch] + 3*pedStds[ch]):
            f.write('\n')
            f.write(t+" "+str(xpos)+" "+str(ypos)+" "+str(zpos)+" "+str(energy/mips[ch]))
        zpos += 2.8125
       
        

        Ch_00_LG  Ch_00_HG  Ch_01_LG  Ch_01_HG  Ch_02_LG  Ch_02_HG  Ch_03_LG  \
0           60.0     189.0      58.0     182.0      59.0     164.0      62.0   
1          186.0    1478.0      66.0     274.0     160.0    1156.0      70.0   
2           63.0     261.0      58.0     237.0      67.0     259.0      71.0   
3          314.0    2768.0      57.0     189.0      57.0     211.0      65.0   
4           60.0     218.0      63.0     238.0      57.0     208.0      65.0   
...          ...       ...       ...       ...       ...       ...       ...   
619694      55.0      89.0      52.0     103.0     107.0     661.0      56.0   
619695     754.0    4044.0      68.0     284.0      69.0     296.0      62.0   
619696      57.0     163.0      54.0     161.0      65.0     257.0      62.0   
619697      59.0      89.0      60.0      86.0      63.0      98.0      61.0   
619698      62.0      63.0      66.0      59.0      60.0      67.0      64.0   

        Ch_03_HG  Ch_04_LG  Ch_04_HG  .